In [1]:
# Povm

In [1]:
import pickle
import time
from typing import List
import datetime as dt
from pathlib import Path

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

from tqdm import tqdm

from quara.data_analysis import data_analysis, physicality_violation_check, report
from quara.objects.composite_system import CompositeSystem
from quara.objects.elemental_system import ElementalSystem
from quara.objects.matrix_basis import get_normalized_pauli_basis
from quara.objects.state import (
    State,
    get_x0_1q,
    get_x1_1q,
    get_y0_1q,
    get_y1_1q,
    get_z0_1q,
    get_z1_1q,
)
from quara.objects.povm import (
    Povm,get_x_measurement
)
from quara.objects.qoperation import QOperation
from quara.protocol.qtomography.standard.standard_povmt import StandardPovmt
from quara.protocol.qtomography.standard.linear_estimator import LinearEstimator
from quara.protocol.qtomography.standard.projected_linear_estimator import (
    ProjectedLinearEstimator,
)

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
start_all = time.time()

In [4]:
# setup system
e_sys = ElementalSystem(0, get_normalized_pauli_basis())
c_sys = CompositeSystem([e_sys])

# |+><+|
state_x0 = get_x0_1q(c_sys)
# |+i><+i|
state_y0 = get_y0_1q(c_sys)
# |0><0|
state_z0 = get_z0_1q(c_sys)
# |1><1|
state_z1 = get_z1_1q(c_sys)
tester_objects = [state_x0, state_y0, state_z0, state_z1]

In [5]:
# Case 1: 軸上
# a0, a1, a2, a3 = 1, 1, 0, 0
# m1 = (1 / np.sqrt(2)) * np.array([a0, a1, a2, a3])
# m2 = (1 / np.sqrt(2)) * np.array([2 - a0, -a1, -a2, -a3])

# Case 2: 軸から一番離れた表面
a0, a1, a2, a3 = 1, 1 / np.sqrt(2), 0, 1 / np.sqrt(2)
m1 = (1 / np.sqrt(2)) * np.array([a0, a1, a2, a3])
m2 = (1 / np.sqrt(2)) * np.array([2 - a0, -a1, -a2, -a3])

# Case 3: 原点
# a0, a1, a2, a3 = 1, 0, 0, 0
# m1 = (1 / np.sqrt(2)) * np.array([a0, a1, a2, a3])
# m2 = (1 / np.sqrt(2)) * np.array([2 - a0, -a1, -a2, -a3])

true_object = Povm(vecs=[m1, m2], c_sys=c_sys)

true_object.vecs

(array([0.70710678, 0.5       , 0.        , 0.5       ]),
 array([ 0.70710678, -0.5       ,  0.        , -0.5       ]))

In [6]:
num_data = [100, 1000]
n_rep = 100
measurement_n = len(true_object.vecs)  # 測定値の数

case_name_list = [
    "LinearEstimator(True)",
    "LinearEstimator(False)",
    "ProjectedLinearEstimator(True)",
    "ProjectedLinearEstimator(False)",
]

seed = 777
qtomography_list = [
    StandardPovmt(
        tester_objects, measurement_n, on_para_eq_constraint=True, seed=seed
    ),
    StandardPovmt(
        tester_objects, measurement_n, on_para_eq_constraint=False, seed=seed
    ),
    StandardPovmt(
        tester_objects, measurement_n, on_para_eq_constraint=True, seed=seed
    ),
    StandardPovmt(
        tester_objects, measurement_n, on_para_eq_constraint=False, seed=seed
    ),
]
para_list = [True, False, True, False]

estimator_list = [
    LinearEstimator(),
    LinearEstimator(),
    ProjectedLinearEstimator(),
    ProjectedLinearEstimator(),
]

estimation_results_list = []
elapsed_times = []

for i, name in enumerate(case_name_list):
    qtomography = qtomography_list[i]
    estimator = estimator_list[i]
    
    start = time.time()
    print(f"Case {i}: {name}")
    print(f"Parametorization: {para_list[i]}")
    print(f"Type of qtomography: {qtomography.__class__.__name__}")
    print(f"Estimator: {estimator.__class__.__name__}")

    estimation_results = data_analysis.estimate(
       qtomography=qtomography,
       true_object=true_object,
       num_data=num_data,
       estimator=estimator,
       iteration=n_rep,
    )
    estimation_results_list.append(estimation_results)
    
    elapsed_time = time.time() - start
    print("elapsed_time:{0}".format(elapsed_time / 60) + "[min]\n")
    elapsed_times.append(elapsed_time)

  5%|▌         | 5/100 [00:00<00:01, 48.12it/s]

Case 0: LinearEstimator(True)
Parametorization: True
Type of qtomography: StandardPovmt
Estimator: LinearEstimator


  7%|▋         | 7/100 [00:00<00:01, 63.78it/s]

elapsed_time:0.02733925183614095[min]

Case 1: LinearEstimator(False)
Parametorization: False
Type of qtomography: StandardPovmt
Estimator: LinearEstimator


  1%|          | 1/100 [00:00<00:17,  5.60it/s]

elapsed_time:0.03000718355178833[min]

Case 2: ProjectedLinearEstimator(True)
Parametorization: True
Type of qtomography: StandardPovmt
Estimator: ProjectedLinearEstimator


  1%|          | 1/100 [00:00<00:15,  6.58it/s]

elapsed_time:0.21584128141403197[min]

Case 3: ProjectedLinearEstimator(False)
Parametorization: False
Type of qtomography: StandardPovmt
Estimator: ProjectedLinearEstimator


100%|██████████| 100/100 [00:13<00:00,  7.42it/s]

elapsed_time:0.22467091878255208[min]



In [7]:
report.export_report("sample_povmt.pdf",
    estimation_results_list,
    case_name_list,
    estimator_list,
    true_object,
    tester_objects,
    seed=seed,
    computation_time=sum(elapsed_times)
)

​Generating table of computation time ...
​Generating table of experimental conditions ...
Generating case list ...


100%|██████████| 100/100 [00:00<00:00, 106050.67it/s]

​​Generating MSE of empirical distributions blocks ...


​​Generating consictency test blocks ...
​Generating a graph for MSE ...


/Users/tomoko/project/rcast/workspace/quara/venv/lib/python3.8/site-packages/numpy/core/_methods.py:216: RuntimeWarning:

Degrees of freedom <= 0 for slice

/Users/tomoko/project/rcast/workspace/quara/venv/lib/python3.8/site-packages/numpy/core/_methods.py:209: RuntimeWarning:

invalid value encountered in double_scalars

100%|██████████| 100/100 [00:00<00:00, 25917.96it/s]

​​Generating physicality violation test blocks ...



  0%|          | 0/2 [00:00<?, ?it/s]
2it [00:00, 65.28it/s]

2it [00:00, 58.71it/s]
  0%|          | 0/2 [00:00<?, ?it/s]
2it [00:00, 32.84it/s]

2it [00:00, 65.72it/s]
  0%|          | 0/2 [00:00<?, ?it/s]
2it [00:00, 59.56it/s]

2it [00:00, 59.11it/s]
  0%|          | 0/2 [00:00<?, ?it/s]
2it [00:00, 62.24it/s]

2it [00:00, 45.61it/s]
  0%|          | 0/2 [00:00<?, ?it/s]
2it [00:00, 52.40it/s]

2it [00:00, 59.76it/s]
  0%|          | 0/2 [00:00<?, ?it/s]
2it [00:00, 45.21it/s]

2it [00:00, 44.06it/s]
  0%|          | 0/2 [00:00<?, ?it/s]
2it [00:00, 50.60it/s]

2it [00:00, 56.73it/s]
  0%|          | 0/2 [00:00<?, ?it/s]
2it [00:00, 55.59it/s]

2it [00:00, 56.09it/s]
100%|██████████| 100/100 [00:00<00:00, 1101.29it/s]


Converting to PDF report ...
​Deleting temporary files ...
Completed to export pdf. (sample_povmt.pdf)


In [13]:
data_analysis.make_mses_graph_estimation_results(
    estimation_results_list,
    case_name_list,
    true_object,
    estimator_list=estimator_list,
)